<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/02_sam2_evaluados_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR SAM 2.0 CON PROMPTS
================================================================================
Sistema de evaluacion para SAM 2.0 con segmentacion GUIADA POR PROMPTS
optimizada para deteccion de personas en fotografia de retrato.

DIFERENCIAS CON SAM 2.0 AUTOMATICO (02_sam_evaluador.py):
- SAM automatico: Genera todas las mascaras posibles sin guia
- SAM con prompts: Usa informacion adicional (puntos, boxes) para guiar
  la segmentacion hacia las personas especificamente

ESTRATEGIAS DE PROMPTS IMPLEMENTADAS:
1. grid_central: Grid de puntos en zona central (regla de tercios fotografia)
2. saliency_points: Puntos en regiones de alta saliencia visual
3. bbox_heuristic: Bounding boxes estimados para personas centrales
4. combined: Combinacion de grid + saliency para robustez maxima

VENTAJAS DE USAR PROMPTS:
- Mayor precision en deteccion de personas vs objetos
- Reduce falsos positivos (objetos que parecen personas)
- Aprovecha composicion fotografica (personas suelen estar centradas)
- Permite refinamiento iterativo (puntos positivos/negativos)

CONFIGURACIONES DE GENERACION:
1. conservative: Pocos prompts, alta precision (3-5 puntos)
2. moderate: Balance prompts-cobertura (9-12 puntos)
3. aggressive: Maximo numero de prompts (16-25 puntos)

ESTRUCTURA DE SALIDA:
/TFM/2_Modelos/sam2_prompts/{modelo}_{estrategia}_{config}/
├── resultados_{timestamp}.json     # Metricas y comparacion con/sin prompts
├── mascaras/                        # Mascaras NPZ con scores SAM
└── visualizaciones/                 # Comparacion visual prompts vs automatico

Entrada: Imagenes desde /TFM/0_Imagenes/
Salida: JSON, mascaras NPZ y visualizaciones comparativas

Autor: Jesus L.
Proyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Octubre 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR SAM 2.0 CON PROMPTS\n================================================================================\nSistema de evaluacion para SAM 2.0 con segmentacion GUIADA POR PROMPTS\noptimizada para deteccion de personas en fotografia de retrato.\n\nDIFERENCIAS CON SAM 2.0 AUTOMATICO (02_sam_evaluador.py):\n- SAM automatico: Genera todas las mascaras posibles sin guia\n- SAM con prompts: Usa informacion adicional (puntos, boxes) para guiar\n  la segmentacion hacia las personas especificamente\n\nESTRATEGIAS DE PROMPTS IMPLEMENTADAS:\n1. grid_central: Grid de puntos en zona central (regla de tercios fotografia)\n2. saliency_points: Puntos en regiones de alta saliencia visual\n3. bbox_heuristic: Bounding boxes estimados para personas centrales\n4. combined: Combinacion de grid + saliency para robustez maxima\n\nVENTAJAS DE USAR PROMPTS:\n- Mayor precision en deteccion de personas vs objetos\n- Reduce f

In [3]:
# =============================================================================
# SETUP GOOGLE COLAB Y DESCARGA DE CHECKPOINTS
# =============================================================================

import os
from pathlib import Path

# Montar Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("Google Drive montado correctamente")
except:
    IN_COLAB = False
    print("No estamos en Colab")

# Checkpoints (mismos que SAM automatico)
CHECKPOINTS_DIR = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints")
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# URLs oficiales de Meta AI
CHECKPOINTS = {
    'sam2_hiera_tiny.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt',
    'sam2_hiera_small.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt',
    'sam2_hiera_base_plus.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt',
    'sam2_hiera_large.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt',
}

print("="*80)
print("DESCARGANDO CHECKPOINTS SAM 2.0")
print("="*80)

for filename, url in CHECKPOINTS.items():
    checkpoint_path = CHECKPOINTS_DIR / filename

    if checkpoint_path.exists():
        size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
        print(f"[EXISTE] {filename}: {size_mb:.1f} MB")
    else:
        print(f"[DESCARGANDO] {filename}...")
        !wget -q --show-progress {url} -O {checkpoint_path}

        if checkpoint_path.exists():
            size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
            print(f"  Completado: {size_mb:.1f} MB")
        else:
            print(f"  ERROR descargando {filename}")

print("\n" + "="*80)
print(f"Checkpoints listos en: {CHECKPOINTS_DIR}")
print("="*80)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente
DESCARGANDO CHECKPOINTS SAM 2.0
[EXISTE] sam2_hiera_tiny.pt: 148.7 MB
[EXISTE] sam2_hiera_small.pt: 175.8 MB
[EXISTE] sam2_hiera_base_plus.pt: 308.5 MB
[EXISTE] sam2_hiera_large.pt: 856.4 MB

Checkpoints listos en: /content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints


In [4]:
# =============================================================================
# INSTALACION DE DEPENDENCIAS
# =============================================================================

print("Instalando dependencias...")

# Dependencias core
# !pip install -q torch torchvision
# !pip install -q opencv-python matplotlib Pillow
# !pip install -q numpy scipy scikit-image

# # SAM 2.0
# !pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

print("Dependencias instaladas correctamente")

Instalando dependencias...
Dependencias instaladas correctamente


In [5]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import sys
import json
import warnings
import gc
import time
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import ndimage

# scikit-image para deteccion de saliencia
from skimage import filters, feature, color

# SAM 2.0 - IMPORTANTE: Usar SAM2ImagePredictor para prompts
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

warnings.filterwarnings('ignore')

# Configurar dispositivo
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("Usando CPU (sera mas lento)")

print("Librerias importadas correctamente")

GPU disponible: Tesla T4
VRAM total: 14.74 GB
Librerias importadas correctamente


In [6]:
# =============================================================================
# CONFIGURACION DE MODELOS
# =============================================================================

@dataclass
class ModeloSAMInfo:
    """Informacion completa de un modelo SAM 2.0"""
    nombre: str
    checkpoint: str
    config: str
    parametros_millones: int
    vram_estimada_gb: float
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        """Obtiene nombre corto para archivos"""
        return self.nombre.replace('sam2_hiera_', '')

# Catalogo de modelos SAM 2.0 disponibles
MODELOS_DISPONIBLES = {
    'tiny': ModeloSAMInfo(
        nombre='sam2_hiera_tiny',
        checkpoint='sam2_hiera_tiny.pt',
        config='sam2_hiera_t.yaml',
        parametros_millones=39,
        vram_estimada_gb=2.0,
        descripcion='Modelo ultraligero optimizado para edge computing'
    ),
    'small': ModeloSAMInfo(
        nombre='sam2_hiera_small',
        checkpoint='sam2_hiera_small.pt',
        config='sam2_hiera_s.yaml',
        parametros_millones=46,
        vram_estimada_gb=3.0,
        descripcion='Balance optimo calidad-velocidad para produccion'
    ),
    'base_plus': ModeloSAMInfo(
        nombre='sam2_hiera_base_plus',
        checkpoint='sam2_hiera_base_plus.pt',
        config='sam2_hiera_b+.yaml',
        parametros_millones=80,
        vram_estimada_gb=5.0,
        descripcion='Modelo avanzado para produccion de alta calidad'
    ),
    'large': ModeloSAMInfo(
        nombre='sam2_hiera_large',
        checkpoint='sam2_hiera_large.pt',
        config='sam2_hiera_l.yaml',
        parametros_millones=224,
        vram_estimada_gb=8.0,
        descripcion='Modelo flagship para maxima precision'
    )
}

In [7]:
# =============================================================================
# ESTRATEGIAS DE PROMPTS
# =============================================================================

@dataclass
class EstrategiaPrompts:
    """Configuracion de una estrategia de generacion de prompts"""
    nombre: str
    tipo: str  # 'points', 'boxes', 'combined'
    num_prompts: int
    descripcion: str
    parametros: Dict[str, Any] = field(default_factory=dict)

ESTRATEGIAS_DISPONIBLES = {
    'grid_central_conservative': EstrategiaPrompts(
        nombre='grid_central_conservative',
        tipo='points',
        num_prompts=5,
        descripcion='Grid 3x3 en zona central, conservador',
        parametros={
            'grid_size': 3,
            'area_central': 0.6,  # 60% central de la imagen
            'puntos_negativos': False
        }
    ),
    'grid_central_moderate': EstrategiaPrompts(
        nombre='grid_central_moderate',
        tipo='points',
        num_prompts=9,
        descripcion='Grid 3x3 completo en zona central',
        parametros={
            'grid_size': 3,
            'area_central': 0.7,
            'puntos_negativos': False
        }
    ),
    'grid_central_aggressive': EstrategiaPrompts(
        nombre='grid_central_aggressive',
        tipo='points',
        num_prompts=16,
        descripcion='Grid 4x4 en zona central con puntos negativos',
        parametros={
            'grid_size': 4,
            'area_central': 0.8,
            'puntos_negativos': True  # Añade puntos negativos en esquinas
        }
    ),
    'saliency_conservative': EstrategiaPrompts(
        nombre='saliency_conservative',
        tipo='points',
        num_prompts=5,
        descripcion='Puntos en top-5 regiones de alta saliencia',
        parametros={
            'num_puntos': 5,
            'threshold_percentil': 95,
            'min_distancia': 50
        }
    ),
    'saliency_moderate': EstrategiaPrompts(
        nombre='saliency_moderate',
        tipo='points',
        num_prompts=10,
        descripcion='Puntos en top-10 regiones de alta saliencia',
        parametros={
            'num_puntos': 10,
            'threshold_percentil': 90,
            'min_distancia': 40
        }
    ),
    'bbox_heuristic': EstrategiaPrompts(
        nombre='bbox_heuristic',
        tipo='boxes',
        num_prompts=1,
        descripcion='Bounding box estimado para persona central',
        parametros={
            'aspect_ratio_persona': [0.4, 0.8],  # Ratio tipico altura/anchura
            'area_minima': 0.15,  # 15% de la imagen minimo
            'centrado': True
        }
    ),
    'combined_moderate': EstrategiaPrompts(
        nombre='combined_moderate',
        tipo='combined',
        num_prompts=12,
        descripcion='Grid central + puntos de saliencia',
        parametros={
            'grid_size': 3,
            'area_central': 0.7,
            'num_saliency': 3,
            'puntos_negativos': False
        }
    ),
    'combined_aggressive': EstrategiaPrompts(
        nombre='combined_aggressive',
        tipo='combined',
        num_prompts=20,
        descripcion='Grid 4x4 + saliencia + puntos negativos',
        parametros={
            'grid_size': 4,
            'area_central': 0.8,
            'num_saliency': 4,
            'puntos_negativos': True
        }
    )
}


In [8]:
# =============================================================================
# GENERADOR DE PROMPTS
# =============================================================================

class GeneradorPrompts:
    """
    Genera prompts (puntos, bounding boxes) para guiar SAM 2.0.

    IMPORTANTE: Los prompts se generan basandose en heuristicas fotograficas
    y analisis de saliencia visual para maximizar deteccion de personas.
    """

    @staticmethod
    def generar_grid_central(img_array: np.ndarray,
                            grid_size: int = 3,
                            area_central: float = 0.7,
                            puntos_negativos: bool = False) -> Tuple[np.ndarray, np.ndarray]:
        """
        Genera grid de puntos en la zona central de la imagen.

        En fotografia de retrato, las personas suelen estar en la zona central
        siguiendo la regla de los tercios.

        Args:
            img_array: Imagen como array numpy (H, W, C)
            grid_size: Tamaño del grid (3x3, 4x4, etc)
            area_central: Porcentaje de area central a cubrir
            puntos_negativos: Si True, añade puntos negativos en esquinas

        Returns:
            puntos_positivos: Array (N, 2) con coordenadas [x, y]
            puntos_negativos: Array (M, 2) o None
        """
        h, w = img_array.shape[:2]

        # Calcular area central
        margin_h = int(h * (1 - area_central) / 2)
        margin_w = int(w * (1 - area_central) / 2)

        central_h_start = margin_h
        central_h_end = h - margin_h
        central_w_start = margin_w
        central_w_end = w - margin_w

        # Generar grid
        y_coords = np.linspace(central_h_start, central_h_end, grid_size, dtype=int)
        x_coords = np.linspace(central_w_start, central_w_end, grid_size, dtype=int)

        puntos_pos = []
        for y in y_coords:
            for x in x_coords:
                puntos_pos.append([x, y])

        puntos_pos = np.array(puntos_pos)

        # Puntos negativos en esquinas (background)
        puntos_neg = None
        if puntos_negativos:
            esquinas = [
                [margin_w // 2, margin_h // 2],  # Superior izquierda
                [w - margin_w // 2, margin_h // 2],  # Superior derecha
                [margin_w // 2, h - margin_h // 2],  # Inferior izquierda
                [w - margin_w // 2, h - margin_h // 2]  # Inferior derecha
            ]
            puntos_neg = np.array(esquinas)

        return puntos_pos, puntos_neg

    @staticmethod
    def generar_puntos_saliencia(img_array: np.ndarray,
                                 num_puntos: int = 5,
                                 threshold_percentil: float = 95,
                                 min_distancia: int = 50) -> np.ndarray:
        """
        Genera puntos en regiones de alta saliencia visual.

        La saliencia detecta regiones que destacan visualmente, donde es
        probable que esten las personas en una fotografia.

        Args:
            img_array: Imagen como array numpy (H, W, C)
            num_puntos: Numero de puntos a generar
            threshold_percentil: Percentil para umbralizar saliencia
            min_distancia: Distancia minima entre puntos (pixels)

        Returns:
            puntos: Array (N, 2) con coordenadas [x, y]
        """
        # Convertir a escala de grises
        if len(img_array.shape) == 3:
            img_gray = color.rgb2gray(img_array)
        else:
            img_gray = img_array

        # Calcular saliencia usando Spectral Residual
        # (metodo rapido y efectivo)
        fft = np.fft.fft2(img_gray)
        magnitude = np.abs(fft)
        phase = np.angle(fft)

        log_amplitude = np.log(magnitude + 1e-10)
        avg_log = ndimage.uniform_filter(log_amplitude, size=3)
        spectral_residual = log_amplitude - avg_log

        saliency_fft = np.exp(spectral_residual + 1j * phase)
        saliency_map = np.abs(np.fft.ifft2(saliency_fft))**2

        # Normalizar
        saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-10)
        saliency_map = ndimage.gaussian_filter(saliency_map, sigma=2)

        # Umbralizar
        threshold = np.percentile(saliency_map, threshold_percentil)
        salient_pixels = saliency_map > threshold

        # Encontrar coordenadas de pixels salientes
        y_coords, x_coords = np.where(salient_pixels)

        if len(y_coords) == 0:
            # Si no hay puntos salientes, usar centro
            h, w = img_gray.shape
            return np.array([[w//2, h//2]])

        # Ponderaciones por intensidad de saliencia
        weights = saliency_map[salient_pixels]

        # Seleccionar puntos distribuidos
        puntos = []
        coords = np.column_stack([x_coords, y_coords])

        # Primer punto: maximo de saliencia
        idx_max = np.argmax(weights)
        puntos.append(coords[idx_max])

        # Resto de puntos: maxima saliencia con restriccion de distancia
        for _ in range(num_puntos - 1):
            if len(coords) == 0:
                break

            # Calcular distancias a puntos ya seleccionados
            distancias = np.min([
                np.linalg.norm(coords - p, axis=1)
                for p in puntos
            ], axis=0)

            # Puntos candidatos: lejos de los ya seleccionados
            candidatos = distancias > min_distancia

            if not np.any(candidatos):
                # Si no hay candidatos, relajar restriccion
                min_distancia *= 0.7
                candidatos = distancias > min_distancia

            if not np.any(candidatos):
                break

            # Entre candidatos, elegir el de mayor saliencia
            weights_candidatos = weights[candidatos]
            coords_candidatos = coords[candidatos]

            idx_next = np.argmax(weights_candidatos)
            puntos.append(coords_candidatos[idx_next])

            # Eliminar punto seleccionado
            mask = np.ones(len(coords), dtype=bool)
            mask[candidatos] = False
            mask[np.where(candidatos)[0][idx_next]] = False
            coords = coords[mask]
            weights = weights[mask]

        return np.array(puntos)

    @staticmethod
    def generar_bbox_heuristica(img_array: np.ndarray,
                               aspect_ratio_range: List[float] = [0.4, 0.8],
                               area_minima: float = 0.15,
                               centrado: bool = True) -> np.ndarray:
        """
        Genera bounding box estimado para persona central.

        Usa heuristicas fotograficas: personas suelen ocupar 15-50% de la imagen,
        con aspect ratio tipico 0.4-0.8 (mas altas que anchas), y centradas.

        Args:
            img_array: Imagen como array numpy (H, W, C)
            aspect_ratio_range: Rango [min, max] de aspect ratio (altura/anchura)
            area_minima: Area minima como fraccion de la imagen
            centrado: Si True, centra el bbox

        Returns:
            bbox: Array [x1, y1, x2, y2] en formato xyxy
        """
        h, w = img_array.shape[:2]
        img_area = h * w

        # Area objetivo (entre minima y 50% de la imagen)
        target_area = img_area * np.clip(area_minima + 0.2, area_minima, 0.5)

        # Aspect ratio objetivo (promedio del rango)
        target_aspect = np.mean(aspect_ratio_range)

        # Calcular dimensiones del bbox
        # area = w_box * h_box
        # aspect = h_box / w_box
        # => h_box = aspect * w_box
        # => area = w_box * aspect * w_box = aspect * w_box^2
        # => w_box = sqrt(area / aspect)

        w_box = int(np.sqrt(target_area / target_aspect))
        h_box = int(w_box * target_aspect)

        # Asegurar que cabe en la imagen
        w_box = min(w_box, int(w * 0.9))
        h_box = min(h_box, int(h * 0.9))

        if centrado:
            # Centrar bbox
            x1 = (w - w_box) // 2
            y1 = (h - h_box) // 2
        else:
            # Posicion aleatoria pero razonable
            x1 = int(w * 0.2)
            y1 = int(h * 0.1)

        x2 = x1 + w_box
        y2 = y1 + h_box

        # Asegurar limites
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)

        return np.array([x1, y1, x2, y2])

    @staticmethod
    def generar_prompts(img_array: np.ndarray,
                       estrategia: EstrategiaPrompts) -> Dict[str, Any]:
        """
        Genera prompts segun la estrategia especificada.

        Returns:
            Dict con:
                - 'point_coords': Array (N, 2) o None
                - 'point_labels': Array (N,) con 1=positivo, 0=negativo, o None
                - 'boxes': Array (M, 4) en formato xyxy, o None
                - 'metadata': Info adicional
        """
        resultado = {
            'point_coords': None,
            'point_labels': None,
            'boxes': None,
            'metadata': {
                'estrategia': estrategia.nombre,
                'tipo': estrategia.tipo,
                'num_prompts': estrategia.num_prompts
            }
        }

        if estrategia.tipo == 'points':
            if 'grid' in estrategia.nombre:
                # Grid central
                params = estrategia.parametros
                puntos_pos, puntos_neg = GeneradorPrompts.generar_grid_central(
                    img_array,
                    grid_size=params.get('grid_size', 3),
                    area_central=params.get('area_central', 0.7),
                    puntos_negativos=params.get('puntos_negativos', False)
                )

                if puntos_neg is not None:
                    # Combinar positivos y negativos
                    resultado['point_coords'] = np.vstack([puntos_pos, puntos_neg])
                    labels_pos = np.ones(len(puntos_pos), dtype=int)
                    labels_neg = np.zeros(len(puntos_neg), dtype=int)
                    resultado['point_labels'] = np.concatenate([labels_pos, labels_neg])
                else:
                    resultado['point_coords'] = puntos_pos
                    resultado['point_labels'] = np.ones(len(puntos_pos), dtype=int)

            elif 'saliency' in estrategia.nombre:
                # Puntos de saliencia
                params = estrategia.parametros
                puntos = GeneradorPrompts.generar_puntos_saliencia(
                    img_array,
                    num_puntos=params.get('num_puntos', 5),
                    threshold_percentil=params.get('threshold_percentil', 95),
                    min_distancia=params.get('min_distancia', 50)
                )
                resultado['point_coords'] = puntos
                resultado['point_labels'] = np.ones(len(puntos), dtype=int)

        elif estrategia.tipo == 'boxes':
            # Bounding box heuristico
            params = estrategia.parametros
            bbox = GeneradorPrompts.generar_bbox_heuristica(
                img_array,
                aspect_ratio_range=params.get('aspect_ratio_persona', [0.4, 0.8]),
                area_minima=params.get('area_minima', 0.15),
                centrado=params.get('centrado', True)
            )
            resultado['boxes'] = bbox.reshape(1, 4)  # Shape (1, 4)

        elif estrategia.tipo == 'combined':
            # Combinacion de grid + saliencia
            params = estrategia.parametros

            # Grid central
            puntos_grid, puntos_neg = GeneradorPrompts.generar_grid_central(
                img_array,
                grid_size=params.get('grid_size', 3),
                area_central=params.get('area_central', 0.7),
                puntos_negativos=params.get('puntos_negativos', False)
            )

            # Puntos de saliencia
            puntos_sal = GeneradorPrompts.generar_puntos_saliencia(
                img_array,
                num_puntos=params.get('num_saliency', 3),
                threshold_percentil=92,
                min_distancia=60
            )

            # Combinar
            all_points = [puntos_grid, puntos_sal]
            all_labels = [
                np.ones(len(puntos_grid), dtype=int),
                np.ones(len(puntos_sal), dtype=int)
            ]

            if puntos_neg is not None:
                all_points.append(puntos_neg)
                all_labels.append(np.zeros(len(puntos_neg), dtype=int))

            resultado['point_coords'] = np.vstack(all_points)
            resultado['point_labels'] = np.concatenate(all_labels)

        return resultado

print("Generador de prompts definido")

Generador de prompts definido


In [9]:
# =============================================================================
# PROCESADOR DE IMAGENES CON PROMPTS
# =============================================================================

class ProcesadorImagenesPrompts:
    """
    Procesa imagenes usando SAM 2.0 con prompts para segmentacion guiada.

    DIFERENCIA CLAVE vs automatico:
    - Automatico (SAM2AutomaticMaskGenerator): Genera TODAS las mascaras
    - Prompts (SAM2ImagePredictor): Genera mascaras SOLO donde se indica
    """

    def __init__(self,
                 modelo_info: ModeloSAMInfo,
                 estrategia: EstrategiaPrompts,
                 checkpoints_path: Path,
                 directorio_salida: Path,
                 max_size: int = 1024,
                 generar_visualizaciones: bool = True):
        """
        Args:
            modelo_info: Informacion del modelo SAM
            estrategia: Estrategia de generacion de prompts
            checkpoints_path: Directorio con checkpoints
            directorio_salida: Directorio para resultados
            max_size: Dimension maxima para procesar
            generar_visualizaciones: Si generar visualizaciones
        """
        self.modelo_info = modelo_info
        self.estrategia = estrategia
        self.checkpoints_path = checkpoints_path
        self.directorio_salida = directorio_salida
        self.max_size = max_size
        self.generar_vis = generar_visualizaciones

        self.predictor = None
        self.inicializado = False

        # Crear directorios
        (self.directorio_salida / "mascaras").mkdir(parents=True, exist_ok=True)
        if self.generar_vis:
            (self.directorio_salida / "visualizaciones").mkdir(parents=True, exist_ok=True)

    def inicializar(self):
        """Inicializa el modelo SAM 2.0"""
        if self.inicializado:
            return

        print(f"Inicializando {self.modelo_info.nombre}...")

        checkpoint_path = self.checkpoints_path / self.modelo_info.checkpoint

        if not checkpoint_path.exists():
            raise FileNotFoundError(f"Checkpoint no encontrado: {checkpoint_path}")

        # Construir modelo
        sam2_model = build_sam2(
            config_file=self.modelo_info.config,
            ckpt_path=str(checkpoint_path),
            device=DEVICE
        )

        # IMPORTANTE: Usar SAM2ImagePredictor para prompts
        self.predictor = SAM2ImagePredictor(sam2_model)

        self.inicializado = True
        print(f"Modelo inicializado correctamente")

    def procesar_imagen(self, imagen_path: Path) -> Dict[str, Any]:
        """
        Procesa una imagen con prompts.

        Returns:
            Dict con resultados, metricas y rutas de archivos
        """
        if not self.inicializado:
            self.inicializar()

        tiempo_inicio = time.time()

        # Cargar imagen
        img_pil = Image.open(imagen_path).convert("RGB")
        dim_original = img_pil.size

        # Redimensionar si necesario
        redimensionada = False
        if self.max_size and max(img_pil.size) > self.max_size:
            ratio = self.max_size / max(img_pil.size)
            new_size = tuple(int(d * ratio) for d in img_pil.size)
            img_pil = img_pil.resize(new_size, Image.LANCZOS)
            redimensionada = True

        img_array = np.array(img_pil)
        tiempo_carga = (time.time() - tiempo_inicio) * 1000

        # Generar prompts
        tiempo_prompts_inicio = time.time()
        prompts = GeneradorPrompts.generar_prompts(img_array, self.estrategia)
        tiempo_prompts = (time.time() - tiempo_prompts_inicio) * 1000

        # Setear imagen en el predictor
        tiempo_set_inicio = time.time()
        self.predictor.set_image(img_array)
        tiempo_set = (time.time() - tiempo_set_inicio) * 1000

        # Predecir con prompts
        tiempo_pred_inicio = time.time()

        masks, scores, logits = self.predictor.predict(
            point_coords=prompts['point_coords'],
            point_labels=prompts['point_labels'],
            box=prompts['boxes'],
            multimask_output=True  # SAM genera 3 mascaras candidatas
        )

        tiempo_prediccion = (time.time() - tiempo_pred_inicio) * 1000

        # Filtrar personas (basado en heuristicas)
        mascaras_personas, metadatos_personas = self._filtrar_personas(
            masks, scores, img_array.shape[:2]
        )

        tiempo_total = (time.time() - tiempo_inicio) * 1000

        # Guardar mascaras
        archivos_mascaras = self._guardar_mascaras(
            imagen_path.stem,
            mascaras_personas,
            metadatos_personas
        )

        # Generar visualizacion
        if self.generar_vis and len(mascaras_personas) > 0:
            self._generar_visualizacion(
                imagen_path.stem,
                img_array,
                masks,  # Todas las mascaras generadas
                mascaras_personas,
                metadatos_personas,
                prompts
            )

        # Construir resultado
        resultado = {
            'imagen': imagen_path.name,
            'modelo': self.modelo_info.nombre,
            'estrategia': self.estrategia.nombre,
            'dimensiones': {
                'original': {'width': dim_original[0], 'height': dim_original[1]},
                'procesada': {
                    'width': img_array.shape[1],
                    'height': img_array.shape[0],
                    'redimensionada': redimensionada
                }
            },
            'prompts': {
                'estrategia': self.estrategia.nombre,
                'tipo': self.estrategia.tipo,
                'num_point_coords': len(prompts['point_coords']) if prompts['point_coords'] is not None else 0,
                'num_boxes': len(prompts['boxes']) if prompts['boxes'] is not None else 0,
                'num_puntos_positivos': int(np.sum(prompts['point_labels'] == 1)) if prompts['point_labels'] is not None else 0,
                'num_puntos_negativos': int(np.sum(prompts['point_labels'] == 0)) if prompts['point_labels'] is not None else 0
            },
            'resultados': {
                'mascaras_generadas': len(masks),
                'personas_detectadas': len(mascaras_personas),
                'archivos_mascaras': archivos_mascaras
            },
            'metricas': {
                'scores_sam': [float(s) for s in scores],
                'scores_personas': [float(m['score']) for m in metadatos_personas]
            },
            'procesamiento': {
                'tiempo_carga_ms': round(tiempo_carga, 2),
                'tiempo_generacion_prompts_ms': round(tiempo_prompts, 2),
                'tiempo_set_imagen_ms': round(tiempo_set, 2),
                'tiempo_prediccion_ms': round(tiempo_prediccion, 2),
                'tiempo_total_ms': round(tiempo_total, 2)
            },
            'timestamp': datetime.now().isoformat()
        }

        return resultado

    def _filtrar_personas(self, masks: np.ndarray, scores: np.ndarray,
                         img_shape: Tuple[int, int]) -> Tuple[List[np.ndarray], List[Dict]]:
        """
        Filtra mascaras que probablemente sean personas.

        Usa mismas heuristicas que SAM automatico para consistencia.
        """
        h, w = img_shape
        img_area = h * w

        mascaras_personas = []
        metadatos = []

        for mask, score in zip(masks, scores):
            # Calcular propiedades
            area = int(np.sum(mask))

            if area == 0:
                continue

            area_ratio = area / img_area

            # Bounding box
            ys, xs = np.where(mask)
            if len(ys) == 0:
                continue

            x1, x2 = xs.min(), xs.max()
            y1, y2 = ys.min(), ys.max()
            bbox_w = x2 - x1 + 1
            bbox_h = y2 - y1 + 1

            aspect_ratio = bbox_h / (bbox_w + 1e-10)

            # Centroide
            cy = (y1 + y2) / 2 / h
            cx = (x1 + x2) / 2 / w
            center_distance = np.sqrt((cx - 0.5)**2 + (cy - 0.5)**2)

            # Compacidad
            bbox_area = bbox_w * bbox_h
            compactness = area / (bbox_area + 1e-10)

            # CRITERIOS (mismos que automatico)
            criterios = {
                'aspect_ratio': 0.3 <= aspect_ratio <= 1.5,
                'area': 0.05 <= area_ratio <= 0.9,
                'score': score >= 0.85,
                'centrality': center_distance <= 0.3,
                'compactness': compactness >= 0.4
            }

            # Scoring: minimo 3 de 5 criterios
            criterios_cumplidos = sum(criterios.values())
            person_confidence = criterios_cumplidos / len(criterios)

            if criterios_cumplidos >= 3:
                mascaras_personas.append(mask)
                metadatos.append({
                    'bbox': [int(x1), int(y1), int(x2), int(y2)],
                    'area': area,
                    'area_ratio': float(area_ratio),
                    'aspect_ratio': float(aspect_ratio),
                    'center_distance': float(center_distance),
                    'compactness': float(compactness),
                    'score': float(score),
                    'person_confidence': float(person_confidence),
                    'criterios': criterios,
                    'criterios_cumplidos': criterios_cumplidos
                })

        return mascaras_personas, metadatos

    def _guardar_mascaras(self, imagen_stem: str, mascaras: List[np.ndarray],
                         metadatos: List[Dict]) -> List[str]:
        """Guarda mascaras en formato NPZ comprimido"""
        archivos = []

        for idx, (mask, meta) in enumerate(zip(mascaras, metadatos)):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{imagen_stem}_persona_{idx}_{timestamp}.npz"
            filepath = self.directorio_salida / "mascaras" / filename

            # Guardar mascara + metadatos
            np.savez_compressed(
                filepath,
                mascara=mask.astype(np.uint8),
                bbox=np.array(meta['bbox']),
                area=meta['area'],
                score=meta['score'],
                person_confidence=meta['person_confidence'],
                criterios_cumplidos=meta['criterios_cumplidos']
            )

            archivos.append(f"mascaras/{filename}")

        return archivos

    def _generar_visualizacion(self, imagen_stem: str, img_array: np.ndarray,
                              todas_mascaras: np.ndarray,
                              mascaras_personas: List[np.ndarray],
                              metadatos: List[Dict],
                              prompts: Dict[str, Any]):
        """
        Genera visualizacion 3-panel:
        1. Original con prompts overlayed
        2. Todas las mascaras generadas por SAM
        3. Solo personas filtradas
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Panel 1: Original + Prompts
        axes[0].imshow(img_array)
        axes[0].set_title("Original + Prompts", fontsize=12, fontweight='bold')
        axes[0].axis('off')

        # Dibujar prompts
        if prompts['point_coords'] is not None:
            coords = prompts['point_coords']
            labels = prompts['point_labels']

            # Puntos positivos (verde)
            pos_points = coords[labels == 1]
            if len(pos_points) > 0:
                axes[0].scatter(pos_points[:, 0], pos_points[:, 1],
                              c='lime', s=100, marker='*',
                              edgecolors='darkgreen', linewidths=2,
                              label='Prompts +', zorder=10)

            # Puntos negativos (rojo)
            neg_points = coords[labels == 0]
            if len(neg_points) > 0:
                axes[0].scatter(neg_points[:, 0], neg_points[:, 1],
                              c='red', s=100, marker='x',
                              linewidths=3,
                              label='Prompts -', zorder=10)

        if prompts['boxes'] is not None:
            for box in prompts['boxes']:
                x1, y1, x2, y2 = box
                rect = mpatches.Rectangle(
                    (x1, y1), x2-x1, y2-y1,
                    linewidth=3, edgecolor='cyan',
                    facecolor='none', label='Prompt Box'
                )
                axes[0].add_patch(rect)

        axes[0].legend(loc='upper right', fontsize=10)

        # Panel 2: Todas las mascaras
        axes[1].imshow(img_array)

        # Overlay todas las mascaras con colores
        for idx, mask in enumerate(todas_mascaras):
            color = np.random.random(3)
            overlay = np.zeros_like(img_array, dtype=np.float32)
            # Convertir mascara a bool para indexar correctamente
            mask_bool = mask.astype(bool) if mask.dtype != bool else mask
            overlay[mask_bool] = color
            axes[1].imshow(overlay, alpha=0.4)

        axes[1].set_title(f"Todas las Máscaras ({len(todas_mascaras)})",
                         fontsize=12, fontweight='bold')
        axes[1].axis('off')

        # Panel 3: Solo personas
        axes[2].imshow(img_array)

        for idx, (mask, meta) in enumerate(zip(mascaras_personas, metadatos)):
            # Overlay mascara
            color_mask = np.zeros_like(img_array, dtype=np.float32)
            # Convertir mascara a bool para indexar correctamente
            mask_bool = mask.astype(bool) if mask.dtype != bool else mask
            color_mask[mask_bool] = [0, 1, 0]  # Verde
            axes[2].imshow(color_mask, alpha=0.3)

            # Bounding box
            x1, y1, x2, y2 = meta['bbox']
            rect = mpatches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor='lime',
                facecolor='none'
            )
            axes[2].add_patch(rect)

            # Label con scores
            label = f"#{idx+1}\nConf:{meta['person_confidence']:.2f}\nSAM:{meta['score']:.2f}"
            axes[2].text(
                x1, y1-5, label,
                color='white', fontsize=9,
                bbox=dict(boxstyle='round', facecolor='green', alpha=0.7),
                verticalalignment='bottom'
            )

        axes[2].set_title(f"Personas Detectadas ({len(mascaras_personas)})",
                         fontsize=12, fontweight='bold')
        axes[2].axis('off')

        # Titulo general
        fig.suptitle(
            f"{self.modelo_info.nombre} | {self.estrategia.nombre} | {imagen_stem}",
            fontsize=14, fontweight='bold'
        )

        plt.tight_layout()

        # Guardar
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{imagen_stem}_{timestamp}.png"
        filepath = self.directorio_salida / "visualizaciones" / filename

        plt.savefig(filepath, dpi=150, bbox_inches='tight')
        plt.close(fig)

    def liberar_recursos(self):
        """Libera recursos del modelo"""
        if self.predictor is not None:
            del self.predictor
            self.predictor = None

        Utilidades.liberar_memoria()
        self.inicializado = False

print("Procesador de imagenes con prompts definido")

Procesador de imagenes con prompts definido


In [10]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares"""

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def obtener_memoria_gpu() -> Dict:
        """Info de memoria GPU"""
        if not torch.cuda.is_available():
            return {'disponible': False}
        return {
            'disponible': True,
            'nombre': torch.cuda.get_device_name(0),
            'asignada_mb': round(torch.cuda.memory_allocated(0) / 1024**2, 2),
            'reservada_mb': round(torch.cuda.memory_reserved(0) / 1024**2, 2)
        }

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """Guarda JSON"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)


In [11]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """Gestiona checkpoint para procesamiento incremental"""

    def __init__(self, ruta_checkpoint: Path):
        self.ruta = ruta_checkpoint
        self.completadas = set()
        self.cargar()

    def cargar(self):
        """Carga checkpoint existente"""
        if self.ruta.exists():
            with open(self.ruta, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.completadas = set(data.get('completadas', []))

    def marcar_completada(self, nombre_imagen: str):
        """Marca imagen como completada"""
        self.completadas.add(nombre_imagen)
        self.guardar()

    def esta_completada(self, nombre_imagen: str) -> bool:
        """Verifica si imagen esta completada"""
        return nombre_imagen in self.completadas

    def guardar(self):
        """Guarda checkpoint"""
        data = {
            'completadas': sorted(list(self.completadas)),
            'total': len(self.completadas),
            'ultima_actualizacion': datetime.now().isoformat()
        }
        Utilidades.guardar_json(data, self.ruta)

    def obtener_estadisticas(self) -> Dict:
        """Obtiene estadisticas del checkpoint"""
        return {
            'total_completadas': len(self.completadas),
            'imagenes': sorted(list(self.completadas))
        }


In [12]:
# =============================================================================
# EVALUADOR PRINCIPAL
# =============================================================================

class EvaluadorSAM2Prompts:
    """Evaluador principal con prompts"""

    def __init__(self,
                 ruta_imagenes: Path,
                 ruta_salida_base: Path,
                 ruta_checkpoints: Path,
                 modelos_evaluar: List[str],
                 estrategias_evaluar: List[str],
                 max_size: int = 1024,
                 generar_visualizaciones: bool = True,
                 pausa_entre_imagenes: float = 2.0):
        self.ruta_imagenes = ruta_imagenes
        self.ruta_salida_base = ruta_salida_base
        self.ruta_checkpoints = ruta_checkpoints
        self.modelos_evaluar = modelos_evaluar
        self.estrategias_evaluar = estrategias_evaluar
        self.max_size = max_size
        self.generar_vis = generar_visualizaciones
        self.pausa = pausa_entre_imagenes

    def evaluar_combinacion(self, modelo_key: str, estrategia_key: str):
        """Evalua una combinacion modelo + estrategia"""
        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        estrategia = ESTRATEGIAS_DISPONIBLES[estrategia_key]

        modelo_nombre = modelo_info.obtener_nombre_sanitizado()

        # Directorio: modelo_estrategia
        config_id = f"{modelo_nombre}_{estrategia.nombre}"
        dir_salida = self.ruta_salida_base / config_id

        print(f"\n{'='*80}")
        print(f"EVALUANDO: {config_id}")
        print(f"{'='*80}")

        # Checkpoint
        checkpoint_path = dir_salida / "checkpoint.json"
        checkpoint = GestorCheckpoint(checkpoint_path)

        print(f"Imagenes ya completadas: {checkpoint.obtener_estadisticas()['total_completadas']}")

        # Obtener imagenes
        extensiones = ['*.jpg', '*.JPG', '*.png', '*.PNG', '*.jpeg', '*.JPEG']
        imagenes = []
        for ext in extensiones:
            imagenes.extend(self.ruta_imagenes.glob(ext))
        imagenes = sorted(set(imagenes))

        # Filtrar pendientes
        imagenes_pendientes = [
            img for img in imagenes
            if not checkpoint.esta_completada(img.name)
        ]

        print(f"Total imagenes: {len(imagenes)}")
        print(f"Pendientes: {len(imagenes_pendientes)}")

        if not imagenes_pendientes:
            print("Todas las imagenes ya fueron procesadas")
            return

        # Inicializar procesador
        print(f"\nInicializando procesador...")
        procesador = ProcesadorImagenesPrompts(
            modelo_info=modelo_info,
            estrategia=estrategia,
            checkpoints_path=self.ruta_checkpoints,
            directorio_salida=dir_salida,
            max_size=self.max_size,
            generar_visualizaciones=self.generar_vis
        )

        procesador.inicializar()

        print(f"\nProcesando {len(imagenes_pendientes)} imagenes...")

        # Guardar resultados consolidados
        resultados_todos = []

        # Procesar imagenes
        for idx, imagen_path in enumerate(imagenes_pendientes, 1):
            print(f"\n[{idx}/{len(imagenes_pendientes)}] {imagen_path.name}")

            try:
                resultado = procesador.procesar_imagen(imagen_path)

                print(f"  Prompts: {resultado['prompts']['num_point_coords']} puntos")
                print(f"  Personas: {resultado['resultados']['personas_detectadas']}")
                print(f"  Tiempo: {resultado['procesamiento']['tiempo_total_ms']:.1f} ms")

                # Añadir a consolidado
                resultados_todos.append(resultado)

                # Guardar JSON individual
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                json_filename = f"{imagen_path.stem}_{timestamp}.json"
                json_path = dir_salida / "json" / json_filename
                json_path.parent.mkdir(parents=True, exist_ok=True)
                Utilidades.guardar_json(resultado, json_path)

                # Marcar como completada
                checkpoint.marcar_completada(imagen_path.name)
                print(f"  Checkpoint actualizado")

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

            finally:
                Utilidades.liberar_memoria()
                if idx < len(imagenes_pendientes):
                    time.sleep(self.pausa)

        # Guardar consolidado
        if resultados_todos:
            consolidado = {
                'modelo': modelo_info.nombre,
                'estrategia': estrategia.nombre,
                'total_imagenes': len(resultados_todos),
                'timestamp': datetime.now().isoformat(),
                'resultados': resultados_todos
            }

            timestamp_consolidado = datetime.now().strftime("%Y%m%d_%H%M%S")
            json_consolidado = f"consolidado_{timestamp_consolidado}.json"
            Utilidades.guardar_json(
                consolidado,
                dir_salida / json_consolidado
            )
            print(f"\nJSON consolidado guardado: {json_consolidado}")

        # Limpiar
        print(f"\nLiberando recursos...")
        procesador.liberar_recursos()
        del procesador
        Utilidades.liberar_memoria()

        print(f"\n{'='*80}")
        print(f"COMBINACION COMPLETADA: {config_id}")
        print(f"{'='*80}")

    def ejecutar_evaluacion_completa(self):
        """Ejecuta evaluacion completa de todas las combinaciones"""
        print(f"\n{'#'*80}")
        print("INICIO DE EVALUACION SAM 2.0 CON PROMPTS")
        print(f"{'#'*80}")
        print(f"\nConfiguracion:")
        print(f"  Modelos: {self.modelos_evaluar}")
        print(f"  Estrategias: {self.estrategias_evaluar}")
        print(f"  Imagenes: {self.ruta_imagenes}")
        print(f"  Salida: {self.ruta_salida_base}")

        total_combinaciones = len(self.modelos_evaluar) * len(self.estrategias_evaluar)
        combinacion_actual = 0

        for modelo_key in self.modelos_evaluar:
            for estrategia_key in self.estrategias_evaluar:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                try:
                    self.evaluar_combinacion(modelo_key, estrategia_key)

                except Exception as e:
                    print(f"\nERROR FATAL: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    print(f"\nContinuando con siguiente combinacion...")
                    continue

                finally:
                    Utilidades.liberar_memoria()
                    time.sleep(5)

        print(f"\n{'#'*80}")
        print("EVALUACION COMPLETA FINALIZADA")
        print(f"{'#'*80}")

print("Clase evaluador principal definida")

Clase evaluador principal definida


In [13]:
# =============================================================================
# FUNCION MAIN PARA EJECUCION
# =============================================================================

def main():
    """Funcion principal para ejecutar la evaluacion"""

    print("\n" + "="*80)
    print("CONFIGURACION DEL EVALUADOR SAM 2.0 CON PROMPTS")
    print("="*80)

    # CONFIGURACION DE RUTAS
    ruta_imagenes = Path("/content/drive/MyDrive/TFM/0_Imagenes")
    ruta_salida_base = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2_prompts")
    ruta_checkpoints = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints")

    # CONFIGURACION DE EVALUACION
    modelos_evaluar = ['tiny', 'small', 'base_plus', 'large']

    # Seleccionar estrategias de prompts
    estrategias_evaluar = list(ESTRATEGIAS_DISPONIBLES.keys())

    # Parametros
    max_size = 1024
    generar_visualizaciones = True
    pausa_entre_imagenes = 2.0

    # Validar configuracion
    print("\nValidando configuracion...")
    try:
        if not ruta_imagenes.exists():
            raise FileNotFoundError(f"Imagenes no encontradas: {ruta_imagenes}")

        if not ruta_checkpoints.exists():
            raise FileNotFoundError(f"Checkpoints no encontrados: {ruta_checkpoints}")

        for modelo_key in modelos_evaluar:
            modelo_info = MODELOS_DISPONIBLES[modelo_key]
            checkpoint_path = ruta_checkpoints / modelo_info.checkpoint
            if not checkpoint_path.exists():
                raise FileNotFoundError(f"Checkpoint no encontrado: {checkpoint_path}")

        print("Configuracion validada")

    except Exception as e:
        print(f"ERROR: {str(e)}")
        return

    # Resumen
    print(f"\nRESUMEN:")
    print(f"  Modelos: {modelos_evaluar}")
    print(f"  Estrategias: {estrategias_evaluar}")
    print(f"  Combinaciones: {len(modelos_evaluar) * len(estrategias_evaluar)}")
    print(f"  Dataset: {ruta_imagenes}")
    print(f"  Salida: {ruta_salida_base}")
    print(f"  Checkpoints: {ruta_checkpoints}")

    # Mostrar info de estrategias
    print(f"\nESTRATEGIAS DE PROMPTS:")
    for est_key in estrategias_evaluar:
        est = ESTRATEGIAS_DISPONIBLES[est_key]
        print(f"  - {est.nombre}:")
        print(f"      Tipo: {est.tipo}")
        print(f"      Prompts: {est.num_prompts}")
        print(f"      Descripcion: {est.descripcion}")

    # Crear evaluador
    evaluador = EvaluadorSAM2Prompts(
        ruta_imagenes=ruta_imagenes,
        ruta_salida_base=ruta_salida_base,
        ruta_checkpoints=ruta_checkpoints,
        modelos_evaluar=modelos_evaluar,
        estrategias_evaluar=estrategias_evaluar,
        max_size=max_size,
        generar_visualizaciones=generar_visualizaciones,
        pausa_entre_imagenes=pausa_entre_imagenes
    )

    # Ejecutar
    print(f"\nINICIANDO EVALUACION...")
    evaluador.ejecutar_evaluacion_completa()

    print(f"\n{'='*80}")
    print("EVALUACION FINALIZADA")
    print(f"{'='*80}")

print("Funcion main definida")
print("\n" + "="*80)
print("LISTO PARA EJECUTAR")
print("="*80)

Funcion main definida

LISTO PARA EJECUTAR


In [ ]:
if __name__ == "__main__":
    main()


CONFIGURACION DEL EVALUADOR SAM 2.0 CON PROMPTS

Validando configuracion...
Configuracion validada

RESUMEN:
  Modelos: ['tiny', 'small', 'base_plus', 'large']
  Estrategias: ['grid_central_conservative', 'grid_central_moderate', 'grid_central_aggressive', 'saliency_conservative', 'saliency_moderate', 'bbox_heuristic', 'combined_moderate', 'combined_aggressive']
  Combinaciones: 32
  Dataset: /content/drive/MyDrive/TFM/0_Imagenes
  Salida: /content/drive/MyDrive/TFM/2_Modelos/sam2_prompts
  Checkpoints: /content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints

ESTRATEGIAS DE PROMPTS:
  - grid_central_conservative:
      Tipo: points
      Prompts: 5
      Descripcion: Grid 3x3 en zona central, conservador
  - grid_central_moderate:
      Tipo: points
      Prompts: 9
      Descripcion: Grid 3x3 completo en zona central
  - grid_central_aggressive:
      Tipo: points
      Prompts: 16
      Descripcion: Grid 4x4 en zona central con puntos negativos
  - saliency_conservative:
      Tipo: p